In [1]:
import requests
import time
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [35]:
# Set the path to the Chromedriver
DRIVER_PATH = 'C:/Users/yurek/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'

service = Service(executable_path=DRIVER_PATH)
options = webdriver.ChromeOptions()

# Enable headless mode
options.add_argument('--headless=new')
options.add_argument("--window-size=1920,1200")  # Set the window size

# Initialize the Chrome driver
driver = webdriver.Chrome(service=service, options=options)

URL = 'https://www.sony.co.in/interchangeable-lens-cameras/aps-c-e-mount-mirrorless'
driver.get(URL)

# Wait for content to load (adjust selector to match the page's content)
try:
    WebDriverWait(driver, 100).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "GalleryListItem__ButtonContainer"))
    )
except Exception as e:
    print(f"Error waiting for content: {e}")

# Get the dynamically loaded HTML
time.sleep(2)  # Add extra wait time if needed
webpage = driver.page_source

In [36]:
# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(webpage, 'html.parser')
all_links = soup.find_all('a', class_='GalleryListItem__Button js-datalayer-action-event') 
product_links = [link['href'] for link in all_links]

def correct_links(link):
    if 'https' not in link:
        return 'https://www.sony.co.in' + link
    return link

final_links = [correct_links(link) for link in product_links]
print(final_links)
print(len(final_links))

['https://www.sony.co.in/electronics/interchangeable-lens-cameras/zv-e10m2', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-6700', 'https://www.sony.co.in/interchangeable-lens-cameras/products/zv-e10', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-6600', 'https://www.sony.co.in/interchangeable-lens-cameras/products/ilce-6400', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-6100']
6


In [31]:
final_links.extend(['https://www.sony.co.in/interchangeable-lens-cameras/products/ilme-fx3', 'https://www.sony.co.in/interchangeable-lens-cameras/products/ilce-1', 'https://www.sony.co.in/interchangeable-lens-cameras/products/ilce-7c', 'https://www.sony.co.in/interchangeable-lens-cameras/products/ilce-7sm3', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-9m2', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-6600', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-7rm2', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-7rm4a', 'https://www.sony.co.in/interchangeable-lens-cameras/products/ilce-6400', 'https://www.sony.co.in/interchangeable-lens-cameras/products/ilce-7m3-body-kit', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-7sm2', 'https://www.sony.co.in/electronics/interchangeable-lens-cameras/ilce-7rm4'])

In [4]:
# Function to extract product title

def get_title(soup):
    
    try:
        # Outer tag object 
        title = soup.find('div', class_='product-model p5').find('span')

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [5]:
# Function to extract Product Price

def get_model(soup):
    
    try:
        models = soup.find_all('p', class_='mlp-model-no')
        models_value = [model.text for model in models]
        model_list = [model_value.strip() for model_value in models_value]

    except AttributeError:
        model_list = []
    
    return model_list

In [27]:
def get_description(soup):

    product_description = []
    
    try:
        description = soup.find('div', class_='desc p3')
        if description is not None:
            product_description = [des.text for des in description]
            product_description = [item for item in product_description if not str(item).isdigit()]

    except AttributeError:
        product_description = []
    
    return product_description

In [7]:
def get_card_highlights(soup):
    
    try:
        # Outer tag object 
        card_highlights = soup.find('h1', class_='t6 product-title')

        # Inner NavigatableString Object
        card_highlights = card_highlights.text

        # Title as a string value
        card_highlights = card_highlights.strip()

    except AttributeError:
        card_highlights = ""

    return card_highlights

In [8]:
def get_combo(soup):
    
    try:
        sizes = soup.find_all('p', class_='mlp-diff t6')
        sizes_value = [size.text for size in sizes]
        size_list = [size_value.strip() for size_value in sizes_value]

    except AttributeError:
        size_list = []
    
    return size_list

In [9]:
def get_price(soup):

    try:
        prices = soup.find_all("p", class_='price p1')
        prices_value = [price.text for price in prices]
        price_list = [price_value.strip() for price_value in prices_value]

    except AttributeError:
        price_list = []

    return price_list

In [10]:
def get_rating(soup):
    
    try:
        # Outer tag object 
        review = soup.find('div', class_='product-rating')

        # Inner NavigatableString Object
        review_value = review.get('data-stars')

        # Title as a string value
        review_string = review_value.strip()

    except AttributeError:
        review_string = ""

    return review_string

In [11]:
def get_review_count(soup):
    
    try:
        # Outer tag object 
        count = soup.find('span', class_='review-count').text
        # Inner NavigatableString Object
        count_value = count #.get('data-review-count')

        # Title as a string value
        count_string = count_value.strip()

    except AttributeError:
        count_string = ""
    
    return count_string

In [12]:
def get_product_images(soup, driver):

    try:
        # Outer tag object
        buy_link = soup.find('a', 'btn btn-large btn-block buy buy-button retailer_btn-align').get('href')
        buy_link = 'https:' + buy_link
        driver.get(buy_link)
        buy_page = driver.page_source
        image_soup = BeautifulSoup(buy_page, 'html.parser')

        images = image_soup.find_all('img', class_='0 iq-img')
        image_links = [image.get('src') for image in images]
    
    except AttributeError:
        image_links = []

    return image_links

In [13]:
def get_product_features(soup):

    try:
        # Outer tag object
        features = soup.find_all('p', class_='copy ghost-center with-icon')
        feature_list = [feature.text for feature in features]

    except AttributeError:
        feature_list = []

    return feature_list

In [14]:
def get_product_specifications(soup):
    
    try:
        spec_key = soup.find_all('dt', class_='l3')
        spec_key_list = [key.text for key in spec_key]

        spec_value = soup.find_all('dd', class_='p3')
        spec_value_list = [value.text for value in spec_value]

        spec = dict(zip(spec_key_list, spec_value_list))

    except AttributeError:
        spec = {}

    return spec    

In [39]:
data = {'name': [], 'model': [], 'combo': [], 'price': [], 'rating': [], 'review_count': [], 'images': [], 'highlights': [], 'description': [], 'specification': [], 'features': [], 'sub_category': 'APS-C E-mount Mirrorless', 'category': 'Interchangeable-lens Cameras'}

for link in final_links:
    driver.get(link)
    new_webpage = driver.page_source
    new_soup = BeautifulSoup(new_webpage, 'html.parser')

    # The function call to display all the product information
    data['name'].append(get_title(new_soup))
    data['model'].append(get_model(new_soup))
    data['combo'].append(get_combo(new_soup))
    data['price'].append(get_price(new_soup))
    data['rating'].append(get_rating(new_soup))
    data['review_count'].append(get_review_count(new_soup))
    data['images'].append(get_product_images(new_soup, driver=driver))
    data['highlights'].append(get_card_highlights(new_soup))
    data['description'].append(get_description(new_soup))
    data['specification'].append(get_product_specifications(new_soup))
    data['features'].append(get_product_features(new_soup))

In [40]:
sony_df = pd.DataFrame.from_dict(data)
sony_df['name'].replace('', np.nan, inplace=True)
sony_df = sony_df.dropna(subset=['name'])
sony_df

,name,model,combo,price,rating,review_count,images,highlights,description,specification,features,sub_category,category
0,ZV-E10M2,"[ZV-E10M2, ZV-E10M2K]","[Body Only, Body + 16–50 mm Power Zoom Lens (S...","[Rs. 94,9901, Rs. 104,9901, Rs. 94,9901]",5.0,1,[https://www.sony.co.in/image/a93579ec54e94ebf...,ZV-E10 II vlog camera,"[With a large APS-C 26.0-megapixel, Exmor R C...",{'NUMBER OF PIXELS (Total)': 'Approx. 27.0 meg...,[APS-C format Exmor R CMOS sensor with 26.0 me...,APS-C E-mount Mirrorless,Interchangeable-lens Cameras
1,ILCE-6700,"[ILCE-6700, ILCE-6700L, ILCE-6700M]","[Body Only, Body + 16–50 mm Power Zoom Lens, B...","[Rs. 136,9901, Rs. 136,9901, Rs. 147,4901, Rs....",4.5,67,[https://www.sony.co.in/image/847f8a541fe42c56...,α6700 Premium E-mount APS-C Camera,[The α6700 represents a new generation of APS-...,"{'SENSOR TYPE': 'APS-C type (23.3 x 15.5 mm), ...",[APS-C format back-illuminated Exmor R CMOS se...,APS-C E-mount Mirrorless,Interchangeable-lens Cameras
3,ILCE-6600 / ILCE-6600M,"[ILCE-6600, ILCE-6600M]","[Body Only, Body + 18–135 mm Zoom Lens]","[Rs. 124,9901, Rs. 160,9901, Rs. 124,9901]",4.5,42,[https://www.sony.co.in/image/7824183c1ae26922...,Alpha 6600 premium E-mount APS-C camera,[The α6600 packs great performance and feature...,"{'Lens compatibility': 'Sony E-mount lenses', ...","[Fast 0.02-second23 AF, with 425 phase-detecti...",APS-C E-mount Mirrorless,Interchangeable-lens Cameras
5,ILCE-6100 / ILCE-6100L / ILCE-6100Y,"[ILCE-6100, ILCE-6100L, ILCE-6100Y]","[Body Only, Body + 16–50 mm Power Zoom Lens, B...","[Rs. 64,9901, Rs. 64,9901, Rs. 75,9901, Rs. 89...",4.0,30,[https://www.sony.co.in/image/d23c6dfe4d9809eb...,Alpha 6100 APS-C camera with fast AF,[With its extraordinary AF speed and performan...,"{'Lens compatibility': 'Sony E-mount lenses', ...","[Fast 0.02 sec.4 AF, with 425 phase-detection ...",APS-C E-mount Mirrorless,Interchangeable-lens Cameras


In [21]:
driver.get(final_links[1])
response = driver.page_source
soup1 = BeautifulSoup(response, 'html.parser')
title = get_title(soup1)
print(title)
model = get_model(soup1)
print(model)
combo = get_combo(soup1)
print(combo)
price = get_price(soup1)
print(price)
rating = get_rating(soup1)
print(rating)
count = get_review_count(soup1)
print(count)
image_links = get_product_images(soup1, driver=driver)
print(image_links)
card_highlights = get_card_highlights(soup1)
print(card_highlights)
product_description = get_description(soup1)
print(product_description)
feature_list = get_product_features(soup1)
print(feature_list)
specification = get_product_specifications(soup1)
print(specification)

ZV-E10M2
['ZV-E10M2', 'ZV-E10M2K']
['Body Only', 'Body + 16–50 mm Power Zoom Lens (SELP16502)']
['Rs. 94,9901', 'Rs. 104,9901', 'Rs. 94,9901']
5.0
1
['https://www.sony.co.in/image/a93579ec54e94ebff6fde632089f1fab?fmt=png-alpha&wid=676&hei=400', 'https://www.sony.co.in/image/9d01ff8460e78ad073cf2196596f1e98?fmt=png-alpha&wid=676&hei=400', 'https://www.sony.co.in/image/126e44f6c91914f6cae6816d1ecfddde?fmt=png-alpha&wid=676&hei=400', None, None, None]
ZV-E10 II vlog camera
['With a large APS-C 26.0-megapixel', ' Exmor R CMOS sensor, the latest BIONZ XR image processing engine', ', dedicated vlog settings, and interchangeable lenses, the ZV-E10 II offers unprecedented expressive freedom. Lightweight, portable and boasting a high-capacity NP-FZ100 battery and 5 GHz', ' Wi-Fi connectivity, it’s the ideal choice for vloggers on the go.']
['APS-C format Exmor R CMOS sensor with 26.0 megapixels2 ', 'BIONZ XR image processing engine for stunning 4K footage at up to 60p4 ', 'Creative Look and Cin

In [ ]:
sony_df.to_csv("C:/Users/yurek/Downloads/sony_cameras_interchangeable-lens-cameras.csv", header=True, index=False)